In [2]:
import tensorflow as tf
import os 
from tensorflow.keras.models import load_model

os.path.join('Dataset')
from google.colab import drive
drive.mount("/content/gdrive") 

new_model = tf.keras.models.load_model("/content/gdrive/MyDrive/Kadir/models/imgclass.h5")

new_model.summary()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      9280      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 64)        

In [3]:
import cv2
import matplotlib.pyplot as plt

In [5]:
from tensorflow.keras.preprocessing import image #Burada tensorflow içinde bulunan image'i import ediyoruz

In [6]:
data = tf.keras.utils.image_dataset_from_directory('/content/gdrive/MyDrive/Dataset') #Kullanılacak verisetinin bulunduğu dosyaya erişerek içerik hakkında bilgi alıyoruz.

Found 9605 files belonging to 2 classes.


In [7]:
len(data)

301

In [8]:
data_iterator = data.as_numpy_iterator()  # Burada verisetinin ögeleri numpy ögesine dönüştürülüyor.
batch = data_iterator.next() #Burada sürekli olarak bir sonraki elemana geçiş yapmak için next fonksiyonu kullanılıyor.
batch[0].shape #ögenin boyut bilgisini öğrenmek için kullanılıyor.

(32, 256, 256, 3)

In [9]:
batch[1] #Burada 0 ve 1'lerden oluşan array yazdırılıyor.

array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0], dtype=int32)

In [10]:
data = data.map(lambda x,y:(x/255,y))   #Burada veriler kullanıma daha uygun olacak şekilde yeniden boyutlandırılıyor.
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [11]:
batch[0].max() #Burada kontrol amaçlı yeni boyutlandırmada bulunan en yüksek değer yazdırılıyor.

1.0

In [17]:
train_size = int(len(data)* .70)+1  #Burada veriler train,test ve validation olarak 3 gruba ayrıldığında sahip olacakları büyüklükler belirleniyor.
test_size = int(len(data)*0.01)
val_size = int(len(data)*0.2)

train_size + test_size + val_size

274

In [18]:
train = data.take(train_size) #Burada gruplara yukarıdan belirlenmiş boyutlarına uygun olacak şekilde veriler dağıtılıyor.
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)
print(test_size)

3


In [19]:
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy
pre = Precision()
re = Recall()
acc = BinaryAccuracy()
res = []
for batch in test.as_numpy_iterator():
  X,y = batch
  yhat = new_model.predict(X)
  pre.update_state(y,yhat)
  re.update_state(y,yhat)
  acc.update_state(y,yhat)
  res.append(tf.math.confusion_matrix(y,yhat.round()))

1/1 [==============================] - 0s 44ms/step


In [20]:
print("re: ",re.result().numpy())
print("pre: ",pre.result().numpy())
print("acc: ",acc.result().numpy())
print("Confusion Matrix")
print(sum(res))

re:  0.86538464
pre:  0.9375
acc:  0.8958333
Confusion Matrix
tf.Tensor(
[[41  3]
 [ 7 45]], shape=(2, 2), dtype=int32)
